In [4]:
# 

import pandas as pd
import json

max_ = 50
df_script = pd.DataFrame(columns=('file.no','emotion','valence','arousal','script','script_start','script_end')) 
count=0
for i in range(1, max_ + 1):
    with open(f'./clip/clip_{i}/clip_{i}.json', 'r', encoding = 'utf8') as f:
        data = json.load(f)
        file_no = i
        frames = []
        for f_num in range(len(data['data'])):
            try:
                frame = data['data'][str(f_num + 1)]
                frames.append(frame)
            except:
                pass
            for obj_num in range(len(frame)):
                try:
                    obj_num = str(obj_num)
                    script_start=frames[f_num][obj_num]['text']['script_start']
                    script_end=frames[f_num][obj_num]['text']['script_end']
                    emotion=frames[f_num][obj_num]['emotion']['sound']['emotion']
                    valence=frames[f_num][obj_num]['emotion']['sound']['valence']
                    arousal=frames[f_num][obj_num]['emotion']['sound']['arousal']
                    script=frames[f_num][obj_num]['text']['script']
                    df_script.loc[count]=[file_no,emotion,valence,arousal,script,script_start,script_end]
                    count+=1
                except:
                    pass


# df=df_script.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)
df = df_script.drop_duplicates(keep = 'first', ignore_index = True)
df

,file.no,emotion,valence,arousal,script,script_start,script_end
0,1,surprise,4,2,"야, 여기 있던 내 옷 못 봤어?",31,131
1,1,dislike,3,2,무슨 옷 말하는 건데?,141,194
2,1,surprise,4,1,"그거, 내가 생일에 친구한테 선물받은 원피스!",211,302
3,1,surprise,4,6,친구한테… 선물 받은 거?,331,416
4,1,happy,5,1,"그래, 그 남색에, 어깨 부분에 장식 달린 거 있잖아.",423,519
...,...,...,...,...,...,...,...
513,50,angry,4,4,"왜 그렇게 생각해. 솔직히 그 자식 맨날 바람피고, 언니 데려다가 예비시댁이다 뭐다...",721,968
514,50,dislike,4,3,뭘 세상 다 끝난것처럼 이러고 있어..,1224,1301
515,50,dislike,4,5,야! 좀 그냥 위로해주면 어디 덧나니? 너는 꼭 매사에 그렇게 똑똑한척 잘난척이라 ...,1308,1570
516,50,dislike,4,2,"똑똑한척이 아니라, 결혼을 안하는게 똑똑한거야 요새는.",1572,1705


In [8]:
# clip.no 컬럼 추가

import warnings
warnings.filterwarnings('ignore')

for i in range(1, max_ + 1):
    a = len(df.loc[df['file.no'] == i, 'file.no'])
    df.loc[df['file.no'] == i, 'clip.no'] = [f'{i}_{j}' for j in range(1, a+1)]
df.to_csv('음성데이터.csv')
df

,file.no,emotion,valence,arousal,script,script_start,script_end,clip.no
0,1,surprise,4,2,"야, 여기 있던 내 옷 못 봤어?",31,131,1_1
1,1,dislike,3,2,무슨 옷 말하는 건데?,141,194,1_2
2,1,surprise,4,1,"그거, 내가 생일에 친구한테 선물받은 원피스!",211,302,1_3
3,1,surprise,4,6,친구한테… 선물 받은 거?,331,416,1_4
4,1,happy,5,1,"그래, 그 남색에, 어깨 부분에 장식 달린 거 있잖아.",423,519,1_5
...,...,...,...,...,...,...,...,...
513,50,angry,4,4,"왜 그렇게 생각해. 솔직히 그 자식 맨날 바람피고, 언니 데려다가 예비시댁이다 뭐다...",721,968,50_3
514,50,dislike,4,3,뭘 세상 다 끝난것처럼 이러고 있어..,1224,1301,50_4
515,50,dislike,4,5,야! 좀 그냥 위로해주면 어디 덧나니? 너는 꼭 매사에 그렇게 똑똑한척 잘난척이라 ...,1308,1570,50_5
516,50,dislike,4,2,"똑똑한척이 아니라, 결혼을 안하는게 똑똑한거야 요새는.",1572,1705,50_6


In [ ]:
# 영상파일에서 음성만 추출

from moviepy.editor import *

for i in range(1, 401):
    video = VideoFileClip(f'./clip/clip_{i}/clip_{i}.mp4')
    video.audio.write_audiofile(f'./clip/clip_{i}/clip_{i}.wav')

In [6]:
# 대사별 음성 추출

import librosa
import librosa.display
import soundfile as sf

for i in range(1, max_ + 1):
    y, sr = librosa.load(f'./clip/clip_{i}/clip_{i}.wav', sr = 44100)
    vf = len(y)/sr * 30
    starts = df.loc[df['file.no'] == i, 'script_start']
    ends = df.loc[df['file.no'] == i, 'script_end']
    c_nums = df.loc[df['file.no'] == i, 'clip.no']
    for fr_s, fr_e, c_num in zip(starts, ends, c_nums):
        c_start = round(len(y) * fr_s / vf)
        c_end = round(len(y) * fr_e /vf)
        sf.write(f'./clip_wav/clip{c_num}.wav', y[c_start:c_end], 44100, 'pcm_16')

In [ ]:
import os
import pandas as pd
import librosa
path='./clip_wav/'
file_list=os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.wav')]
music_df=pd.read_csv('음성데이터.csv',index_col=0)
count=0
music=pd.DataFrame(columns=['Mel Spectrogram'])
for i in file_list_py:
    y,sr = librosa.load(path+i,sr=44100)
    mel=librosa.feature.melspectrogram(y=y, sr=sr)
    music.loc[count] = [mel]
    count += 1
music_df=pd.concat([music_df,music],axis=1)
music_df